In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [25]:
#Import data frame
path1 = r"/Users/shuhui_zhao/Desktop/credit_card_approval_prediction/credit_record.csv"
path2 = r"/Users/shuhui_zhao/Desktop/credit_card_approval_prediction/application_record.csv"
#
credit_df=pd.read_csv(path1)
#clients' application info
app_df=pd.read_csv(path2)

In [26]:
print("Number of unique clients in credit_record dataset: {}".format(len(credit_df.ID.unique())))
#To see if there is any missing data in this dataset
credit_df_drop = credit_df.dropna() 
if credit_df.shape==credit_df_drop.shape:
    print ("There is no missing data.")
else:
    print ("There are {} rows with missing data".format(len(credit_df.ID.unique())-len(credit_df_drop.ID.unique())))

Number of unique clients in application_record dataset: 45985


In [6]:
#sort the data by customers' ID
credit_df.sort_values('ID')
#Check duplicates
#For each id, if there are two identical months, delete both observations
#temporary data frame: groups_month, groups_month_count
groups_month = credit_df.groupby(['ID','MONTHS_BALANCE'])
groups_month_count = groups_month.count()
groups_month_count = groups_month_count.rename(columns = {"STATUS":"COUNT"})
if groups_month_count.loc[groups_month_count['COUNT']>1].empty:
    print ("There is no duplicates")
else:
    print ("There are duplicates")


(1048575, 3)
Empty DataFrame
Columns: [COUNT]
Index: []


In [27]:
#reshape data by 'ID' based on 'MONTHS_BALANCE'
credit_pivot = credit_df.pivot(index = 'ID', columns = 'MONTHS_BALANCE', values = 'STATUS')
#print(credit_pivot.head())
#assume that the minimum value of 'MONTHS_BALANCE' is the open_month
#Similarly, assume that the maximum value of 'MONTHS_BALANCE' is the close_month
#group the credit_df by 'ID' and extract the minimum & maximum value of 'MONTHS_BALANCE'
credit_grouped = credit_df.groupby('ID')
credit_pivot['open_month'] = credit_grouped['MONTHS_BALANCE'].min()
credit_pivot['close_month'] = credit_grouped['MONTHS_BALANCE'].max()
credit_pivot['ID'] = credit_pivot.index
credit_pivot = credit_pivot[['ID','open_month','close_month']]
#to merge the data, 'ID' cannot be an index level
credit_pivot.reset_index(drop=True, inplace=True)
#calculate the window_month
credit_pivot["window_month"] = credit_pivot['close_month']-credit_pivot['open_month']
#merge two data frame (LEFT OTTER JOIN)
credit = pd.merge(credit_df, credit_pivot, how = 'left', on = 'ID')
#month on balance
credit['MOB']=credit['MONTHS_BALANCE']-credit['open_month']
credit.sort_values(['ID','MOB'],inplace=True)

#divide 48 months into 4 years
bins = [k for k in range(-61,1,12)]
credit['open_year'] = pd.cut(credit['open_month'],bins,labels = [-5,-4,-3,-2,-1])#open_year_to_today

#save credit data frame in credit0 (in case)
credit0 = credit


In [ ]:
#Count unique ID
de = credit.groupby('open_year').agg({'ID':pd.Series.nunique})
de = de.rename(columns = {"ID":"open_sum"})
de.reset_index(inplace=True)
print(de.head())

In [ ]:
###calculate the cumulative percentage of bad customers (cumulative charge-off rate/cumulative loss rate)

#A customer will be considered as 'bad' (1) if overdue
credit['overdue']=np.where((credit['STATUS']=='2')|(credit['STATUS']=='3')
                           |(credit['STATUS']=='4')|(credit['STATUS']=='5'),1,0)

In [10]:
#According to IFRS 9, consider 12 months as performance window
credit = credit[credit['window_month']>12]
#print(credit.head())

In [11]:
vintage = credit_wm12.groupby(['open_year','MOB']).agg({'ID':pd.Series.nunique})
vintage.reset_index(inplace=True)
vintage['overdue_count'] = np.nan
vintage = vintage[['open_year','MOB','overdue_count']]
vintage = pd.merge(vintage,de,how = 'left',on='open_year')

              ID  MONTHS_BALANCE STATUS  open_month  close_month  \
4        5001712               0      C         -18            0   
5        5001712              -1      C         -18            0   
6        5001712              -2      C         -18            0   
7        5001712              -3      C         -18            0   
8        5001712              -4      C         -18            0   
...          ...             ...    ...         ...          ...   
1048570  5150487             -25      C         -29            0   
1048571  5150487             -26      C         -29            0   
1048572  5150487             -27      C         -29            0   
1048573  5150487             -28      C         -29            0   
1048574  5150487             -29      C         -29            0   

         window_month open_year  
4                  18        -2  
5                  18        -2  
6                  18        -2  
7                  18        -2  
8            

In [32]:
#loop to count overdue
# for i in range(-60,1):
#     ls = [] #to save 
#     for j in range(0,61):
#         #save the ID number
#         due = list(credit[(credit['open_month']==i) & (credit['MOB']==j) & (credit['overdue']==1)]['ID'])
#         ls.extend(due)
#         # calculate non-duplicate ID numbers
#         vintage.loc[(vintage['open_month'] == i) & (vintage['MOB'] == j),'overdue_count'] = len(set(ls))

In [18]:
#loop to count overdue
for i in range(-5,-1):
    ls = [] 
    for j in range(0,61):
        #save the ID number
        due = list(credit[(credit.open_year == i) & (credit.MOB==j) & (credit.overdue==1)]['ID'])
        ls.extend(due)
        # calculate non-duplicate ID numbers
        vintage.loc[(vintage.open_year == i) & (vintage.MOB == j),'overdue_count'] = len(set(ls))

In [19]:
vintage['rate'] = vintage['overdue_count']/vintage['open_sum']
#For the clients have been opened their credit cards for less than 1 year, 
#it is not possible for them to have observe window time more than 12 months
vintage = vintage[vintage['open_year']<-1]
vintage_tb = vintage.pivot(index='open_year',columns='MOB',values='rate')
vintage_tb

    open_year  MOB  overdue_count  open_sum      rate
0          -5    0            0.0      5112  0.000000
1          -5    1            3.0      5112  0.000587
2          -5    2           11.0      5112  0.002152
3          -5    3           19.0      5112  0.003717
4          -5    4           32.0      5112  0.006260
..        ...  ...            ...       ...       ...
300        -1   56            NaN         0       NaN
301        -1   57            NaN         0       NaN
302        -1   58            NaN         0       NaN
303        -1   59            NaN         0       NaN
304        -1   60            NaN         0       NaN

[305 rows x 5 columns]


In [1]:
# plot vintage line chart
plt.rcParams['figure.facecolor'] = 'white'
#replace NaN into 0
vintage_graph = vintage_tb.replace(0,np.nan)
lst = [i for i in range(0,61)]
vintage_plt = vintage_tb[lst].T.plot(legend = True, grid = True, title = 'Cumulative Charge-off Rate (> 60 Days Overdue)')
plt.axvline(30)
#plt.axvline(25)
#plt.axvline(20)
plt.xlabel('Months on Books')
plt.ylabel('Cumulative Charge-off Rate')
plt.show()


NameError: name 'plt' is not defined

In [ ]:
#Define "good" and "bad" situations
#From vintage analysis, we can see that for most cases, 


In [ ]:
#
print(app_df.shape)
app_df.head()

In [ ]:
#Join two data frames by ID
sum_df = pd.concat([credit_df,app_df])
#Sort the merged data frame by ID

